In [1]:
from tqdm import tqdm
from glob import glob
import json

In [8]:
hansard = glob('/home/ubuntu/server2/ssd3/hansard/hansard.jsonl*.splitted.requested')
len(hansard)

5

In [7]:
!head -n 5 {english_texts[0]}

{"src": "\nDR. 14.3.2023 140 \n\n \n\u25a01920 \n\nKod itu memang diberikan oleh Kementerian Kewangan, selalunya kerana agensi-\n\nagensi ini bertukar-tukar mengikut kementerian. Dahulu ia dekat JPM, sebelum itu ia dekat \n\nKementerian Ekonomi. Jadi setiap kali bertukar, maka kod itu perlu diubah dan sebab itu ada \n\nkod itu ia kekal dan ditunjuk berasingan. Jadi, dengan cepat Tuan Pengerusi, saya bacakan. Kita \n\nbermula dengan IRDA dahulu ya. \n\nProjek yang melibatkan peruntukan RM185 juta bagi tahun 2023 ini ialah kerja-kerja \n\npembinaan jalan utama bagi pembangunan Ladang Air Manis sepanjang enam kilometer. Taman \n\nBelia Iskandar Malaysia. Jadi, ini menjawab juga pertanyaan daripada saya ingat Yang \n\nBerhormat Tebrau tadi dan juga sedikit sebanyak Yang Berhormat Pasir Gudang, adakah projek-\n\nprojek di bawah IRDA contohnya hanya bersifat komersial dan pembangunan semata-mata dan \n\napakah ada kesan kepada kemakmuran.  \n\nSelain daripada projek-projek yang berbentuk den

In [9]:
data = []
for f in hansard:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            data.append({
                'ms': l['src'],
                'en': l['r']['result']
            })

27971it [00:03, 8765.72it/s] 
28150it [00:02, 9979.93it/s] 
28083it [00:02, 10051.87it/s]
25223it [00:02, 9993.25it/s] 
28314it [00:02, 10115.97it/s]


In [10]:
len(data)

137741

In [13]:
data[1]['en']

"Dr. 14.3.2023 141\n\n \nTech Park so that the capacity of the Kulim Hi-Tech Park can be further expanded in the surrounding area.\n\nThe same is true in other areas.\n\n  It's just a challenge from year to year, because of the area below\n\nthese corridors are large. If you take the example of the corridor in Sarawak, one Sarawak is indeed\n\nthe focus is on renewable energy but the area is quite large. The same\n\nfor example with the East Coast Corridor, it covers Kelantan, Terengganu, Pahang and Johor.\n\nSo, the difference in terms of economic activity in Pahang with Kelantan and Johor is quite a bit\n\ndifferent However, God willing, we will focus so that there is coordination with\n\neconomic themes and patterns that catalyze better growth. If\n\nI can connect quickly because it's only seven minutes.\n\nIn the Iskandar Region for example, this year we will also have a feasibility study\n\nmodern agriculture in Iskandar Malaysia, especially for the northern Johor area. Plan study

In [14]:
# !pip3 install transformers -U
from transformers import AutoTokenizer, AutoModel
import torch

In [15]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-en')

In [16]:
model = AutoModel.from_pretrained('BAAI/bge-large-en')
_ = model.cuda()

2023-09-13 16:57:01.186763: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 16:57:01.265722: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-13 16:57:01.702694: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-13 16:57:01.702751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [17]:
def embedding(encoded_input):
    encoded_input = {'input_ids': encoded_input.unsqueeze(0)}
    for k in encoded_input:
        encoded_input[k] = encoded_input[k].cuda()

    model_output = model(**encoded_input)
    sentence_embeddings = model_output[0][:, 0]
    v = sentence_embeddings[0].detach().cpu().numpy()
    return v

In [18]:
import numpy as np
from itertools import islice

def batched(iterable, n):
    """
    Batch data into tuples of length n. The last batch may be shorter.
    """
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch

def get_tokens(text):
    encoded_input = tokenizer.encode(text,
                             return_tensors='pt')
    return encoded_input[0]

def chunked_tokens(text, chunk_length):
    tokens = get_tokens(text)
    chunks_iterator = batched(tokens, chunk_length)
    yield from chunks_iterator
    
def len_safe_get_embedding(
    text,
    max_tokens=512,
    average=True
):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(text, chunk_length=max_tokens):
        
        chunk = torch.Tensor(chunk).long()
        
        chunk_embeddings.append(embedding(chunk))
        chunk_lens.append(len(chunk))

    if average:
        chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
        chunk_embeddings = chunk_embeddings / np.linalg.norm(chunk_embeddings)
        chunk_embeddings = chunk_embeddings.tolist()
    return chunk_embeddings

len_safe_get_embedding(data[1]['en'], 2)

Token indices sequence length is longer than the specified maximum sequence length for this model (707 > 512). Running this sequence through the model will result in indexing errors


[-0.010807598673268345,
 0.005599070855480067,
 -0.00041131675980811,
 0.015431057065209955,
 -0.016950372814236333,
 -0.023509260932282228,
 -0.016865547247873282,
 0.02026652284850005,
 0.04291401755249985,
 0.015460663089487556,
 0.0340480428706374,
 -0.006376727939753157,
 0.021190515815242825,
 -0.0319193617279151,
 -0.03406940965797435,
 -0.01576939027644345,
 -0.026696076875208183,
 -0.030070285390404877,
 -0.0315420638017213,
 0.004144317414717307,
 -0.010733127077694112,
 0.009373640583771232,
 -0.053307342789967585,
 -0.03528889092456563,
 -0.027458865717137133,
 0.032753117643821765,
 0.023485221765041704,
 0.011882466841326553,
 0.06841538184855545,
 0.058779016549031116,
 -0.019972845389974587,
 -0.029331011234045377,
 0.01641419832323566,
 -0.04945384283293649,
 -0.02803967312536392,
 -0.01680838213918094,
 0.01281299165585762,
 -0.04568098605885771,
 -0.011923014587296782,
 -0.03731415494060441,
 0.032636939127145694,
 -0.009984600974141,
 0.02489073177867562,
 -0.023702

In [19]:
!mkdir hansard

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
instruction = 'Represent this sentence for searching relevant passages: '
examples = [instruction + data[i]['en'] for i in range(10)]

In [21]:
vs = []
for s in examples:
    vs.append(len_safe_get_embedding(s))

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(vs)[1]

array([0.90908745, 1.        , 0.9011755 , 0.91575393, 0.9089671 ,
       0.90344581, 0.87201895, 0.86819308, 0.88314562, 0.86373266])

In [24]:
examples[1], examples[5]

("Represent this sentence for searching relevant passages: Dr. 14.3.2023 141\n\n \nTech Park so that the capacity of the Kulim Hi-Tech Park can be further expanded in the surrounding area.\n\nThe same is true in other areas.\n\n  It's just a challenge from year to year, because of the area below\n\nthese corridors are large. If you take the example of the corridor in Sarawak, one Sarawak is indeed\n\nthe focus is on renewable energy but the area is quite large. The same\n\nfor example with the East Coast Corridor, it covers Kelantan, Terengganu, Pahang and Johor.\n\nSo, the difference in terms of economic activity in Pahang with Kelantan and Johor is quite a bit\n\ndifferent However, God willing, we will focus so that there is coordination with\n\neconomic themes and patterns that catalyze better growth. If\n\nI can connect quickly because it's only seven minutes.\n\nIn the Iskandar Region for example, this year we will also have a feasibility study\n\nmodern agriculture in Iskandar Ma

In [25]:
import os

for i in tqdm(range(len(data))):
    filename = os.path.join('hansard', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    try:
        v = len_safe_get_embedding(instruction + data[i]['en'])
        d = {
            'text': data[i],
            'v': v,
        }
        with open(filename, 'w') as fopen:
            fopen.write(f'{json.dumps(d)}\n')
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████████████| 137741/137741 [2:51:46<00:00, 13.36it/s]


In [26]:
!du -hs hansard

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
4.2G	hansard
